In [ ]:
import sympy

sympy.init_printing()

# SymPy ODE adjoints

In [ ]:
# SymPy for efficient ODEs with adjoints


def fibrosis_ode(y, u):
    k = [None] * 14

    k[0] = 0.9  # proliferation rates: lambda1=0.9/day,
    k[1] = 0.8  # lambda2=0.8/day
    k[2] = 0.3  # mu_1, mu_2, death rates: 0.3/day
    k[3] = 1e6  # carrying capacity: 10^6 cells
    k[4] = 2  # growth factor degradation: gamma=2/day
    k[5] = (
        240 * 1440
    )  # growth factor secretion rates: beta3=240 molecules/cell/min  ---- beta_3
    k[6] = (
        470 * 1440
    )  # beta1=470 molecules/cell/min                                ---- beta_1
    k[7] = (
        70 * 1440
    )  # beta2=70 molecules/cell/min                                 ---- beta_2
    k[8] = (
        940 * 1440
    )  # alpha1=940 molecules/cell/min, endocytosis rate CSF1       ---- alpha_1
    k[9] = (
        510 * 1440
    )  # alpha2=510 molecules/cell/min, endocytosis rate PDGF     ---- alpha_2
    k[10] = 6e8  # #binding affinities: k1=6x10^8 molecules (PDGF)     ---- k_1
    k[11] = 6e8  # k2=6x10^8 (CSF)                                   ---- k_2
    k[12] = 0
    k[13] = 1e6

    # PDGF antibody
    k_pdfg_ab = 1 * 1440  # 1 / (min * molecule)
    pdgf_ab_deg = -k_pdfg_ab * y[3] * u[0]

    # CSF1 antibody
    k_csf1_ab = 1 * 1440  # 1 / (min * molecule)
    csf1_ab_deg = -k_csf1_ab * y[2] * u[1]

    dx = [None] * 4

    dx[0] = y[0] * (
        k[0] * y[3] / (k[10] + y[3]) * (1 - y[0] / k[3]) - k[2]
    )  # Fibrobasts
    dx[1] = y[1] * (k[1] * y[2] / (k[11] + y[2]) - k[2]) + k[12]  # Mph
    dx[2] = (
        csf1_ab_deg + k[6] * y[0] - k[8] * y[1] * y[2] / (k[11] + y[2]) - k[4] * y[2]
    )  # CSF
    dx[3] = (
        pdgf_ab_deg
        + k[7] * y[1]
        + k[5] * y[0]
        - k[9] * y[0] * y[3] / (k[10] + y[3])
        - k[4] * y[3]
    )  # PDGF

    return dx

num_states = 4
num_controls = 2

y = sympy.IndexedBase("y", shape=(num_states,), positive=True, real=True)
a = sympy.IndexedBase("a", shape=(num_states,), real=True)
u = sympy.IndexedBase("u", shape=(num_controls,), positive=True, real=True)

y_vec = sympy.Matrix([y[i] for i in range(num_states)])
a_vec = sympy.Matrix([a[i] for i in range(num_states)])
u_vec = sympy.Matrix([u[i] for i in range(num_controls)])

sympy_ode = sympy.Matrix(fibrosis_ode(y, u))
sympy_ode_jacobian = sympy_ode.jacobian(y_vec)
sympy_ode_grad = sympy_ode.jacobian(u_vec)

sympy_da_dt = a_vec.T * sympy_ode_jacobian
sympy_dl_du = a_vec.T * sympy_ode_grad

In [ ]:
sympy_da_dt[0]

In [ ]:
sympy.cxxcode(sympy_da_dt[0])

In [ ]:
sympy.cxxcode(sympy_dl_du[0])

In [ ]:
def count_flops(expr, visual=False):
    flops_table = {"ADD": 1, "SUB": 1, "NEG": 1, "MUL": 2, "DIV": 4, "POW": 4}

    ops = sympy.count_ops(expr, visual=True)

    flops = 0
    for op in ops.args:
        op = op.args

        if len(op) == 1:
            flops += flops_table[str(op[0])]
        elif len(op) == 2:
            flops += int(op[0]) * flops_table[str(op[1])]

    return flops

In [ ]:
print(count_flops(sympy_da_dt, visual=True))
print(count_flops(sympy.simplify(sympy_da_dt, ratio=1.0, measure=count_flops)))

# Misc. tests

In [ ]:
x, y, z = sympy.symbols("x y z")
sigma, rho, beta = sympy.symbols("sigma rho beta")

def f(x, y, z, sigma, rho, beta):
    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z

    return dx#, dy, dz

sympy_f = f(x, y, z, sigma, rho, beta)
sympy_df_dsigma = sympy.diff(sympy_f, sigma)

sympy.ccode(sympy_df_dsigma, standard="c11")

In [ ]:
x = sympy.IndexedBase("x")
k = sympy.IndexedBase("k")

def f(x, k):
    x, y, z = x[0], x[1], x[2]
    sigma, rho, beta = k[0], k[1], k[2]

    dx = sigma * (y - x)
    dy = x * (rho - z) - y
    dz = x * y - beta * z

    return sympy.Matrix([dx, dy, dz])

sympy_f = f(x, k)
m = sympy_f.jacobian(sympy.Matrix([x[0], x[1], x[2]]))

In [ ]:
sympy.ccode(m, standard="c11")